In [1]:
import pandas as pd
import numpy as np

#import dataframe of cards
cards = pd.read_pickle('./data/cards.pkl')
#fix card codes to match card code format in decks 'cards.#####'
cards['code'] = 'cards.'+cards['code']
#import dataframe of decks
#check to see if all cards are used in a decks, apparently not!
#unused = [x for x in cards['code'] if x not in decks.keys()]
#Remove that card from the dataframe of cards
cards = cards[cards['code'] != 'cards.00012']

decks = pd.read_pickle('./data/decks.pkl')
#drop the columns that I don't care about
decks = decks.drop(['date_update','description','name','tournament_badge','user_id','user_name'],axis=1)
decks = decks.fillna(0)

In [2]:
decks.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35748 entries, 0 to 35747
Columns: 1844 entries, cards.00005 to mwl_code
dtypes: float64(1841), int64(1), object(2)
memory usage: 505.2 MB


In [3]:
#turn all of the deck numbers in np.int8 to reduce file size
decks_float = decks.select_dtypes(include=['float'])
converted = decks_float.astype(np.int8)
decks[converted.columns] = converted

In [4]:
decks.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35748 entries, 0 to 35747
Columns: 1844 entries, cards.00005 to mwl_code
dtypes: int64(1), int8(1841), object(2)
memory usage: 65.8 MB


In [5]:
#make list of idenity cards
identity_cards = cards[cards['type_code'] == 'identity']
#make two new features keep track of the idenity cards and the faction
for ident in identity_cards['code']:
    decks.loc[decks[ident]==1,'identity_card']=ident
    decks.loc[decks[ident]==1,'faction']= cards[cards['code']==ident]['faction_code'].values[0]

In [6]:
#delete old columns that corresponded to identiy cards
decks = decks.drop(identity_cards['code'],axis=1)

In [7]:
#clean up the last few columns types
decks['id']=pd.to_numeric(decks['id'], downcast='integer')
#decks.loc[:,'mwl_code':'faction'] = decks.loc[:,'mwl_code':'faction'].astype('category')
decks['date_creation'] = pd.to_datetime(decks['date_creation'])

In [64]:
#now I want to remove decks that are not really decks...the non-identity ones to start off
decks = decks[decks['faction'] != 'neutral-runner']
decks = decks[decks['faction'] != 'neutral-corp']

MemoryError: 

In [42]:
#now remove the draft format decks
draft_cards = ['cards.00005','cards.00006','cards.00007','cards.00008','cards.00009','cards.00010','cards.00011','cards.000013']
decks = decks[~decks['identity_card'].isin(draft_cards)]

In [58]:
decks[decks['faction'] == 'neutral-corp']

,cards.01002,cards.01003,cards.01004,cards.01005,cards.01006,cards.01007,cards.01008,cards.01009,cards.01010,cards.01011,...,cards.28002,cards.28003,cards.28004,cards.28005,cards.28006,date_creation,id,mwl_code,identity_card,faction


In [59]:
decks.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35291 entries, 0 to 35747
Columns: 1721 entries, cards.01002 to faction
dtypes: datetime64[ns](1), int32(1), int8(1716), object(3)
memory usage: 64.9 MB


In [63]:
#finally we will pickle this as a much easier to use dataframe
cards.to_pickle('cards_processed.pkl')
decks.to_pickle('decks_processed.pkl')

MemoryError: 